GET THE DATA

In [219]:
!wget --no-check-certificate http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz -O /tmp/20news-bydate.tar.gz

--2020-08-11 17:37:13--  http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz
Resolving qwone.com (qwone.com)... 108.20.201.166
Connecting to qwone.com (qwone.com)|108.20.201.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14464277 (14M) [application/x-gzip]
Saving to: ‘/tmp/20news-bydate.tar.gz’

/tmp/20news-bydate. 100%[===================>]  13.79M  6.60MB/s    in 2.1s    

2020-08-11 17:37:15 (6.60 MB/s) - ‘/tmp/20news-bydate.tar.gz’ saved [14464277/14464277]



IMPORT MODULES

In [220]:
import os
import tarfile
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english') 
import operator

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


EXTRACT THE FILE

In [221]:
file_ref = tarfile.open('/tmp/20news-bydate.tar.gz')
file_ref.extractall('/tmp')
file_ref.close()

STORE THE DATA

In [222]:
data = []
for root, dirs, files in os.walk("/tmp/20news-bydate-train", topdown=False):
  for name in files:
    with open(os.path.join(root, name), 'r',encoding="utf8", errors='ignore') as f:
      text = f.read()
      data.append(text)
      f.close()

In [223]:
# data[0]

In [224]:
# len(data)

PREPROCESSING STEPS :


*   Convert into lowercase
*   Keep only text
*   Remove stopwords





In [225]:
def remove_stopwords(text):
  filtered_words = [w for w in text.split()  if ((not w in sw) and (len(w)>1))]
  return " ".join(filtered_words)

def preprocess(text):
  text = list(map(lambda x:x.lower(),text))
  text = list(map(lambda x : re.sub("[^a-zA-Z]", " ",x),text)) 
  text = list(map(remove_stopwords,text))
  return text

PREPROCESS THE DATA

In [226]:
cleaned = preprocess(data)
# cleaned[:10]

CREATE A NESTED DICTIONARY

FIND FREQUENCY OF EACH WORD IN EACH DOCUMENT


In [227]:
def term_doc_frequency(data):
  final_dict = {}
  for i in range(len(data)):
    doc_dict = {} 
    for w in data[i].split():
        try:
            doc_dict[w].add(i)
        except:
            doc_dict[w] = {i}
    for j in doc_dict:
      doc_dict[j] = len(doc_dict[j])
    final_dict[i] = doc_dict
  return final_dict

In [228]:
final_dict = term_doc_frequency(cleaned)

RETRIEVE THE TOP 3 RELEVANT DOCUMENTS

In [229]:
def top_3(data,final_dict,query):
  original_query = [query]
  processed_query = preprocess(original_query)
  query_dict = dict(zip((processed_query[0].split()),[0]*len((processed_query[0].split()))))
  output_dict = {}
  for i in (list(final_dict.keys())):
    doc = {key: final_dict[i][key] for key in final_dict[i].keys() & query_dict.keys()}
    output_dict[i] = sum(list(doc.values()))
  sorted_output_dict = dict(sorted(output_dict.items(), key=operator.itemgetter(1),reverse=True))
  relevant_3 = [data[i] for i in list(sorted_output_dict.keys())[:3]] 
  return relevant_3

HIGHLIGHT THE PART OF THE DOCUMENT WHERE THE SEARCH STRING HAS OCCURED

In [230]:
from termcolor import colored
import string
def color_text(text,words):
  new = []
  text = text.lower().split()
  for t in text:
    word = "".join(re.findall("[a-z]+", t))
    if word in words:
      new.append(colored(t, 'white', 'on_blue'))
    else:
      new.append(t)
  return ' '.join(new)

In [231]:
#@title What would you like to search? 
#amusing atheists and agnostics organization
original_query= 'amusing atheists and agnostics organization' #@param {type:"string"}
query = original_query.lower()
output = top_3(data,final_dict,original_query)

In [233]:
for i in output:
  print(color_text(i,query.split()))
  print('\n')

from: madhaus@netcom.com (maddi hausmann) subject: re: amusing atheists and agnostics organization: society for putting things on top of other things lines: 40 timmbake@mcl.ucsb.edu (bake timmons) writes: > >ok, you have disproved one thing, but you failed to "nail" me. > >see, nowhere in my post did i claim that something _must_ be believed in. here >are the three possibilities: > > 1) god exists. > 2) god does not exist. > 3) i don't know. > >my attack was on strong atheism, (2). since i am (3), i guess by what you said >below that makes me a weak atheist. [snip] >first of all, you seem to be a reasonable guy. why not try to be more honest >and include my sentence afterwards that honest, it just ended like that, i swear! hmmmm...i recognize the warning signs...alternating polite and rude...coming into newsgroup with huge chip on shoulder...calls people names and then makes nice...whirrr...click...whirrr "clam" bake timmons = bill "shit stirrer connor" q.e.d. whirr click whirr...frank